**Description:** We want to use different ML (supervised and unsupervised) approaches to predict churns of subscriptions of the german newspaper “Die Zeit”. Supervised ML methods can be used to predict (classification methods) subscription churns based on the given dataset. Since a large number of features exists, feature selection as part of an extensive EDA is essential. Unsupervised ML methods could be used first to do clustering on the dataset to identify if there are certain “groups of subscribers”, who share certain features. This clustering could be used to investigate subscriber group specific churn mitigation methods (not only writing emails, as given in the dataset description).

**Project Name:** Churn Prediction - Die Zeit

**Team:** Carlotta Ulm, Silas Mederer, Jonas Bechthold

**Date:** 2020-10-26 to 2020-11-27

# Setting up environment and imports

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import math
import itertools
from time import time

# own modules
import eda_methods as eda

# visualization
import seaborn as sns
sns.set(style="white")  
#sns.set_theme()

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
from pandas.plotting import scatter_matrix

# pandas profiling
from pandas_profiling import ProfileReport

# warnings handler
import warnings
warnings.filterwarnings("ignore")

random_state = 100           # Ensures modeling results can be replicated
np.random.seed(42)

# Display Options for pandas
pd.set_option('display.max_columns', None) # Sets maximum columns displayed in tables
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# Variables for plot sizes
matplotlib.rc('font', size=20)          # controls default text sizes
matplotlib.rc('axes', titlesize=16)     # fontsize of the axes title
matplotlib.rc('axes', labelsize=18)    # fontsize of the x and y labels
matplotlib.rc('xtick', labelsize=18)    # fontsize of the tick labels
matplotlib.rc('ytick', labelsize=18)    # fontsize of the tick labels
matplotlib.rc('legend', fontsize=14)    # legend fontsize
matplotlib.rc('figure', titlesize=20)

####################################################
# Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import fbeta_score, accuracy_score, f1_score, recall_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer 
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

#geodaten
import geopandas as gpd

# Business Understanding 

Please check the [wiki page: Business understanding (publishing and news)](https://github.com/jb-ds2020/nf-ds3-capstone-churn-prevention/wiki/Business-understanding-(publishing-and-news)).

## Dataset Description

Let's get an idea about the columns and find out what they mean.

In [ ]:
# new dataframe 2
df = pd.read_csv('data/f_chtr_churn_traintable_nf_2.csv')
df.info()

In [ ]:
df.head()

In [ ]:
df.tail(1)

In [ ]:
df.drop(["Unnamed: 0","auftrag_new_id"], axis=1, inplace=True)

In [ ]:
eda.describe_plus(df).round(2)

## Get an idea of the column names by sampling

In [ ]:
df.sample(2)

## Dataset Overview with pandas profiling

Profile for new dataframe

In [ ]:
# use pandas_profiling to create report
# save html to plots

#profile = ProfileReport(df)
#profile.to_file("plots/pandas_profiling_report_df2.html")

In [ ]:
# include pandas profiling report via html
from IPython.display import IFrame
IFrame(src='plots/pandas_profiling_report_df2.html', width=1000, height=700)

# Data Editing

We reduce the number of cnt_abo maximum to 5 (maximum number of abos per customer, digital, print and magazines), see [Business Understanding](https://github.com/jb-ds2020/nf-ds3-capstone-churn-prevention/wiki/Business-understanding-(publishing-and-news)).

In [ ]:
df = df[df.cnt_abo < 5]

# Data Cleaning
Purpose: Fix the inconsistencies within the data and handle the missing values

In [ ]:
null_rel = round(df.isin([0]).sum() / df.shape[0]*100,2)
null_rel = null_rel.to_frame()
null_rel.rename(columns={0: "zeros %"}, inplace=True)
eda.meta(df).T.join(null_rel).head(5)

In [ ]:
# numeric and categorical features
print(f"shape {df.shape}")
continues = df.select_dtypes(include=['float64','int64'])
print(f"numeric features {len(continues.columns)}")
categorial = df.select_dtypes(include="object")
print(f"object features  {len(categorial.columns)}")

Now we delete the null/missing values.

In [ ]:
df.dropna(subset=['ort','email_am_kunden'], inplace=True)

In [ ]:
null_rel = round(df.isin([0]).sum() / df.shape[0]*100,2)
null_rel = null_rel.to_frame()
null_rel.rename(columns={0: "zeros  %"}, inplace=True)
eda.meta(df).T.join(null_rel).head(5)

## Conclusion

**Dataframe in general**
- The data set is complete and contains 161,760 entries and 169 features. There are some features that we will not be able to use. For example the label "churn" or the feature "date_x", "kuendigungs_eingangs_datum" or "avg_churn" which probably contains the values from the current churn-prediction-model.

**Missings**
- There are only two features with missing values. The feature "kuendigungs_eingangs_datum" is null if there is no notice of termination, so these nulls are correct. The missing values of ort and email_am_kunden have been dropped.

**Dublicates**
- The feature "auftrag_new_id" is the only one with no dublicates. We will drop this, cause all these entries are unique given by the stakeholder to manage the contracts.

**Data types**
- There are 151 numeric features, a lot of them will be categorial. Also we have 18 object types.

**Zero Values**
- As you can see a lot of the features contain high percentage of "0" values. "zon_community" for example is 100.00% altought it has 3 nunique values. This is an example of how little elements have the nunique characteristic.

# Data Exploration - EDA


In [ ]:
def crosstab_evaluation(feature_column,target_column,relative=True):
    '''function to cross evaluate two features by a pandas cross table
    Inputs:
    feature_column: pandas Series of feature
    target_column: pandas Series of target (y-axis value)
    relative: False -> returns absolute values, True -> gives percentages
    
    Return:
    crosstable
    '''
    crosstable = pd.crosstab(feature_column,target_column)
    if relative:
        crosstable = crosstable.div(crosstable.sum(1),axis=0)
    
    return crosstable

def crosstab_barplot(crosstable,labellist,figsize_x=10,figsize_y=7,xlabelname='Default'):
    '''
    Function to plot a pandas crosstable. 
    
    Inputs:
    crosstable: a pandas crosstable
    labellist: a list with the labels of the data
    xlabelname: Name of the x axis feature
    '''
    crosstable.plot(kind='bar', stacked=True,figsize=(figsize_x,figsize_y))
    plt.xlabel(xlabelname)
    plt.ylabel('Frequency')
    
    #plt.title('Title');
    #plt.xticks(np.arange(2), ('60 months', '36 months',), fontsize=20)
    
    L=plt.legend(fontsize=20,loc=(1.04,0.83))
    L.get_texts()[0].set_text(labellist[0])
    L.get_texts()[1].set_text(labellist[1])

## Describe

In [ ]:
eda.describe_plus(df).head()

## Customer/Personal Features

The following feature are considered as personal information and shortly described:
- anrede: Mr, Mrs
- titel: academic title
- plz_1: first letter of zip code
- plz_2: first two letters of zip code
- plz_3: first three letters of zip code
- ort: city
- metropole: is the city a metropolitan city
- land_iso_code: DE, A, CH (Germany, Austria, Switzerland)

### Anrede

In [ ]:
legendlist = ['churn','no churn']

In [ ]:
df.anrede.value_counts().plot(kind='bar');

In [ ]:
anrede_churn = crosstab_evaluation(df.anrede,df.churn)
crosstab_barplot(anrede_churn,['no churn','churn'],xlabelname='Anrede')

We can observe just small differences in the anrede, unknown and male customers seem to have a slightly higher churn probability. 

### Titel

In [ ]:
df.titel.value_counts().plot(kind='bar');

In [ ]:
titel_churn = crosstab_evaluation(df.titel,df.churn)
crosstab_barplot(titel_churn,['no churn','churn'],xlabelname='Titel')

We can observer a smaller churn rate for academic titles compared to no title. The column other title (sonstiger Titel) is quite useless because we don't know hat kind of titles are included in there.

### PLZ 1 

Find the missing number of zip codes:

In [ ]:
df_zip_xx = df[df.plz_1 == 'xx']
print('Number of missing zip codes entries:',df_zip_xx.shape[0])
print('Percentage of missing zip codes in total:',df_zip_xx.shape[0]/df.shape[0])

There are 19788 entries with no zip code, which is about 12 percent. 

In [ ]:
df_zip_xx.ort.value_counts().nlargest(10)

The xx zip codes are all non german cities (e.g. Switzerland), so we could divide the dataset into germany and out of germany areas.

In [ ]:
plz1_churn = crosstab_evaluation(df.plz_1,df.churn)
crosstab_barplot(plz1_churn,['churn','no churn'],xlabelname='PLZ 1')

Simplified geographic features, here the first digit of the zip code are quite uselfull indicators for determine regions with higher churn rate. 

### PLZ 2

In [ ]:
plz2_churn = crosstab_evaluation(df.plz_2,df.churn)
crosstab_barplot(plz2_churn,['churn','no churn'],xlabelname='PLZ 2',figsize_x=25)

Compated to plz_1 we can see more details by this geographic information, the smearing of geographical data is less with two digits and we can have nice insights in the geographical distribution of our churns.

### PLZ 3

In [ ]:
plz3_churn = crosstab_evaluation(df.plz_3,df.churn)
crosstab_barplot(plz3_churn,['no churn','churn'],xlabelname='PLZ 3',figsize_x=50)
plt.xlim(0,50);

By using the 3 digit PLZ we can observe much higher variations in the churn rate. We should definitely include this feature in our model.

Since this feature has a lot of entries, it is usefull to use a nice geographical visualization with geopandas. The link to geopandas for germany can be found here: https://juanitorduz.github.io/germany_plots/

In [ ]:
# geodata
plz_shape_df = gpd.read_file('data/plz-gebiete.shp', dtype={'plz': str})

plz_shape_df.info()

In [ ]:
top_cities = {
    'Berlin': (13.404954, 52.520008), 
    'Cologne': (6.953101, 50.935173),
    'Düsseldorf': (6.782048, 51.227144),
    'Frankfurt am Main': (8.682127, 50.110924),
    'Hamburg': (9.993682, 53.551086),
    'Leipzig': (12.387772, 51.343479),
    'Munich': (11.576124, 48.137154),
    'Dortmund': (7.468554, 51.513400),
    'Stuttgart': (9.181332, 48.777128),
    'Nuremberg': (11.077438, 49.449820),
    'Hannover': (9.73322, 52.37052)
}

In [ ]:
plz_region_df = pd.read_csv(
    'data/zuordnung_plz_ort.csv', 
    sep=',', 
    dtype={'plz': str}
)

plz_region_df.drop('osm_id', axis=1, inplace=True)

plz_region_df.head()

In [ ]:
# Merge data.
germany_df = pd.merge(
    left=plz_shape_df, 
    right=plz_region_df, 
    on='plz',
    how='inner'
)

germany_df.drop(['note'], axis=1, inplace=True)

Mapping Functions to map churn probabilities for each subset of plz digits into a dataframe. 

In [ ]:
def convert_plz_1_to_prob(plz):
    index = str(plz)[0]
    #print(index)
    value = plz1_churn.iloc[int(index),1]
    return value

def convert_plz_2_to_prob(plz):
    index = str(plz)[0:2]
    #print(index)
    value = plz2_churn[plz2_churn.index == index].iloc[0,1]
    #print(value)
    return value

def convert_plz_3_to_prob(plz):
    index = str(plz)[0:3]
    #print(index)
    value = plz3_churn[plz3_churn.index == index].iloc[0,1]
    #print(value)
    return value

In [ ]:
germany_df['churn_plz_1'] = germany_df.plz.apply(lambda x: convert_plz_1_to_prob(x))

In [ ]:
germany_df['churn_plz_2'] = germany_df.plz.apply(lambda x: convert_plz_2_to_prob(x))

In [ ]:
germany_df['churn_plz_3'] = germany_df.plz.apply(lambda x: convert_plz_3_to_prob(x))

Include the number of inhabitants in the dataframe:

In [ ]:
plz_einwohner_df = pd.read_csv(
    'data/plz_einwohner.csv', 
    sep=',', 
    dtype={'plz': str, 'einwohner': int}
)

plz_einwohner_df.head()

In [ ]:
# Merge data.
germany_df = pd.merge(
    left=germany_df, 
    right=plz_einwohner_df, 
    on='plz',
    how='left'
)

#germany_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(11,16))

germany_df.plot(
    ax=ax, 
    column='churn_plz_1', 
    categorical=False, 
    legend=True, 
    cmap='jet',
    alpha=0.8,

)

for c in top_cities.keys():

    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )
    
ax.set(
    title='Germany: Churn Rate of first digit plz', 
    aspect=1.5, 
    facecolor='lightblue'
);
fig.savefig('plots/churn_rate_landscape_plz_1_digit.png',dpi=300)

We can now visually see the higher churn rate in the eastern part of Germany. 

In [ ]:
fig, ax = plt.subplots(figsize=(11,16))

germany_df.plot(
    ax=ax, 
    column='churn_plz_2', 
    categorical=False, 
    legend=True, 
    cmap='jet',
    alpha=0.8,

)

for c in top_cities.keys():

    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )
    
ax.set(
    title='Germany: Churn Rate of first two digit plz', 
    aspect=1.5, 
    facecolor='lightblue'
);
fig.savefig('plots/churn_rate_landscape_plz_2_digit.png',dpi=300)

Now we can more clearly see some smaller regions with much lower and much higher churn rates.

In [ ]:
fig, ax = plt.subplots(figsize=(11,16))

germany_df.plot(
    ax=ax, 
    column='churn_plz_3', 
    categorical=False, 
    legend=True, 
    cmap='jet',
    alpha=0.8,

)

for c in top_cities.keys():

    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )
    
ax.set(
    title='Germany: Churn Rate of first three digit plz', 
    aspect=1.5, 
    facecolor='lightblue'
);
fig.savefig('plots/churn_rate_landscape_plz_3_digit.png',dpi=300)

This gives us a very detailled overview of the geographical distribution of the churn rate, with much less flattening of the data. We can observe areas with almost zero churn rate and regions with more than 60 percent churn rate.  

### Metropole

In [ ]:
metropole_churn = crosstab_evaluation(df.metropole,df.churn)
crosstab_barplot(metropole_churn,['no churn','churn'],xlabelname='metropole')

No significant influence if either a metropolitan city or not is found. 

### Land iso code

In [ ]:
land_iso_churn = crosstab_evaluation(df.land_iso_code,df.churn)
crosstab_barplot(land_iso_churn,['no churn','churn'],xlabelname='land')

Germany has the highest churn rate, people from abroad (Austria (AT), Switzerland (CH) and other countries abroad) tend to churn with a lower probability. People from abroad who are subscribers tend to be more commited to their subscription. 

### Ort/City

In [ ]:
a = df.groupby('ort').size().nlargest(20)
a.plot(kind='bar',ylabel='number of subscribers',figsize=(11,6));

The Zeit Top City is Berlin, followed by Hamburg and Munich. Vienna surpsisingly has a quite high number of subscribers of about 4000.

In [ ]:
b = list(a.index)
df_top_cities = df[df['ort'].apply((lambda x: x in b))]

ort_churn = crosstab_evaluation(df_top_cities.ort,df_top_cities.churn)
crosstab_barplot(ort_churn,['no churn','churn'],xlabelname='ort')

The churn rate varies over the twenty largest cities. 

### Summary Customer Related Features

To sum up, the following was found about the customer related features:

- **anrede**: Small differences: keep feature.
- **titel**: Smaller churn rate for academic titles compared to no title: keep feature
- **plz_1, plz_2, plz_3**: The plz code gives us a very detailled overview of the geographical distribution of the churn rate: keep feature  
- **ort:** Geographic information, but similar to plz: should be dropped
- **metropole:** Just slight difference in churn rate: keep feature
- **land_iso_code:** There is a country influence on the churn rate: keep feature

## Subscription Features - Jonas

The following features are related to the subscription kind and shortly described:

- kanal
- objekt_name 
- aboform_name
- zahlung_rhythmus_name
- rechnungsmonat
- zahlung_weg_name 
- studentenabo
- unterbrechung

### Kanal

In [ ]:
df.kanal.value_counts().plot(kind='bar');

In [ ]:
kanal_churn = crosstab_evaluation(df.kanal,df.churn)
# sort crosstable by churn probability
#kanal_churn.sort_values(by=1,ascending=True,inplace=True)
crosstab_barplot(kanal_churn,['no churn','churn'],xlabelname='kanal')

In [ ]:
kanal_churn = crosstab_evaluation(df.kanal,df.churn)
# sort crosstable by churn probability
kanal_churn.sort_values(by=1,ascending=True,inplace=True)
crosstab_barplot(kanal_churn,['no churn','churn'],xlabelname='kanal')

The channel of the subrscitpion is an important feature for churn. The churn rate increases from less than 20 percent for mailings to almost 40 percent for SEA (Search Engine Advertising).  

### cnt_abo

In [ ]:
fig, ax = plt.subplots(figsize=(11,16))
ax = sns.countplot(x="cnt_abo", hue='churn',data=df)
#ax.set_xlim(0,10)

### objekt_name

In [ ]:
df.objekt_name.value_counts().plot(kind='bar',figsize=(11,6));

In [ ]:
objekt_churn = crosstab_evaluation(df.objekt_name,df.churn)
objekt_churn.sort_values(by=1,ascending=True,inplace=True)
crosstab_barplot(objekt_churn,['no churn','churn'],xlabelname='objekt name')

Combined subscription of Die Zeit with Christ & Welt has a remarkably smaller churn rate than Zeit Digital and Die Zeit print. 

In [ ]:
fig, ax = plt.subplots(figsize=(11,16))
ax = sns.countplot(x="objekt_name", hue='churn',data=df)

### aboform_name

In [ ]:
df.aboform_name.value_counts().plot(kind='bar',figsize=(11,6));

In [ ]:
aboform_churn = crosstab_evaluation(df.aboform_name,df.churn)
aboform_churn.sort_values(by=1,ascending=True,inplace=True)
crosstab_barplot(aboform_churn,['no churn','churn'],xlabelname='aboform')

Also here, we can see that Prämienabo has a much smaller churn rate than others, in particular Geschenkabo and Negative Option. 

### zahlung_rhythmus_name

In [ ]:
df.zahlung_rhythmus_name.value_counts().plot(kind='bar',figsize=(11,6));

In [ ]:
fig, ax = plt.subplots(figsize=(16,16))
ax = sns.countplot(x='zahlung_rhythmus_name',data=df,hue='churn')
ax.set_xlabel('');

In [ ]:
zahlung_rythmus_churn = crosstab_evaluation(df.zahlung_rhythmus_name,df.churn)
zahlung_rythmus_churn.sort_values(by=1,inplace=True,ascending=True)
crosstab_barplot(zahlung_rythmus_churn,['no churn','churn'],xlabelname='zahlung rythmus')

We can observe a strong dependence of the payment period on the churn rate. Payments with less than one year period tend to have higher churn rates.

### rechnungsmonat

In [ ]:
df.rechnungsmonat.value_counts().plot(kind='bar',figsize=(11,6));
plt.xlabel('rechnungsmonat');

Rechnungsmonat? What is the meaning?

In [ ]:
rechnungsmonat_churn = crosstab_evaluation(df.rechnungsmonat,df.churn)
crosstab_barplot(rechnungsmonat_churn,['no churn','churn'],xlabelname='rechnungsmonat')

If there is a billing months, a subscriber has a higher tendency to churn the subscription than without a billing month. 

### zahlung_weg_name

In [ ]:
df.zahlung_weg_name.value_counts().plot(kind='bar',figsize=(11,6));

In [ ]:
zahlungsweg_churn = crosstab_evaluation(df.zahlung_weg_name,df.churn)
crosstab_barplot(zahlungsweg_churn,['no churn','churn'],xlabelname='zahlungsweg')

If the payment is made by direct debit (Bankeinzug), the churn rate is much lower than if the payment is made by invoice. Credit Cards and Paypal are in between.

### studentenabo

In [ ]:
df.studentenabo.value_counts().plot(kind='bar',figsize=(11,6));

In [ ]:
studentenabo_churn = crosstab_evaluation(df.studentenabo,df.churn)
crosstab_barplot(studentenabo_churn,['no churn','churn'],xlabelname='studenten abo')
plt.ylabel('probability');

The students tend to churn more frequently.

#### Studentenmap

In [ ]:
plz3_students = crosstab_evaluation(df.plz_3,df.studentenabo)
#crosstab_barplot(plz3_students,['no student abo','student abo']ticklabelname='PLZ 3',figsize_x=50)

In [ ]:
def convert_plz_3_to_student(plz):
    index = str(plz)[0:3]
    #print(index)
    value = plz3_students[plz3_students.index == index].iloc[0,1]
    #print(value)
    return value

In [ ]:
germany_df['studenten_plz_3'] = germany_df.plz.apply(lambda x: convert_plz_3_to_student(x))

In [ ]:
fig, ax = plt.subplots(figsize=(11,16))

germany_df.plot(
    ax=ax, 
    column='studenten_plz_3', 
    categorical=False, 
    legend=True, 
    cmap='jet',
    alpha=0.8,

)

for c in top_cities.keys():

    ax.text(
        x=top_cities[c][0], 
        y=top_cities[c][1] + 0.08, 
        s=c, 
        fontsize=12,
        ha='center', 
    )

    ax.plot(
        top_cities[c][0], 
        top_cities[c][1], 
        marker='o',
        c='black', 
        alpha=0.5
    )
    
ax.set(
    title='Germany: Student Abo Rate of first three digit plz', 
    aspect=1.5, 
    facecolor='lightblue'
);
fig.savefig('plots/student_abo_rate.png',dpi=300)

Here we can see where the student churn rate is the highest and where it is the lowest.

### unterbrechung

In [ ]:
df.unterbrechung.value_counts().plot(kind='bar',figsize=(11,6));
plt.xlabel('unterbrechung');

In [ ]:
unterbrechung_churn = crosstab_evaluation(df.unterbrechung,df.churn)
crosstab_barplot(unterbrechung_churn,['no churn','churn'],xlabelname='unterbrechung')
plt.ylabel('probability');

The churn rate for interruptions of the subscription seems to be the same.  

### Summary Subscription Data

To sum up, the following was found about the subscription related features:

- **kanal:** An important feature for churn (20 to almost 40 percent churn rate for different channels: keep feature  
- **objekt_name:** Combined subscription have much less churn than digital and print subscriptions: keep feature  
- **aboform_name:** Churn rate is dependent on the form of subscription: keep feature 
- **zahlung_rhythmus_name:** Dependence of the payment period on the churn rate: keep feature
- **rechnungsmonat:** If there is a billing months, a subscriber has a higher tendency to churn: keep feature
- **zahlung_weg_name:** Direct debit (Bankeinzug) churn rate is much lower than if the payment is made by invoice: keep feature 
- **studentenabo:** Students tend to churn more frequently: keep feature
- **unterbrechung:** The churn rate for interruptions of the subscription seems to be the same: could be kept!

## Time/Temporal Features - Jonas

The following features are related to time subscription kind and shortly described: 
- lesedauer
- liefer_beginn_evt
- abo_registrierung_min
- nl_registrierung_min
- date_x
- kuendigungs_eingangs_datum


### lesedauer

In [ ]:
df.lesedauer.value_counts().sort_index().plot(kind='bar',figsize=(25,6));
#df.lesedauer.value_counts().sort_values('lesedauer').plot(kind='bar',figsize=(25,6));
plt.xlabel('lesedauer');

In [ ]:
lesedauer_churn = crosstab_evaluation(df.lesedauer,df.churn)
crosstab_barplot(lesedauer_churn,['no churn','churn'],xlabelname='lesedauer',figsize_x=25,figsize_y=10)
plt.ylabel('churn probability');
plt.xticks(np.arange(0, 88, 6.0),rotation=0);

The lesedauer is an important measure for the churn probability. With fewer months of lesedauer, the churn rate decreases, and then after 12 months there is a wave pattern with a period of 12 months (24, 36, 38, 60 months and so on) in which the churn rate increases.

In [ ]:
fig, ax = plt.subplots(figsize=(25,10))
ax = sns.countplot(x='lesedauer',data=df,hue='churn')
ax.set_xlabel('');
plt.xticks(np.arange(0, 88, 6.0));
plt.xlabel('lesedauer');

### liefer_beginn_evt

In [ ]:
df.liefer_beginn_evt.isna().sum()

In [ ]:
# convert liefer beginn to datetime
df.liefer_beginn_evt.head()

In [ ]:
df['liefer_beginn_evt'] = pd.to_datetime(df['liefer_beginn_evt'])
#df['liefer_beginn_evt'] = df['liefer_beginn_evt'].dt.date
df.liefer_beginn_evt.describe()

In [ ]:
df.liefer_beginn_evt.hist(bins=100,figsize=(11,6));
plt.xlabel('Time');
plt.ylabel('Count');

This feature can be kept, since all users have this feature included. 

In [ ]:
#plt.subplots(figsize=(15,6))
#ax = sns.countplot(x='liefer_beginn_evt', data=df)
#ax.set(xlabel='liefer_beginn_evt', ylabel='Frequency')
#ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

### abo_registrierung_min

In [ ]:
df.abo_registrierung_min.isna().sum()

In [ ]:
# convert abo registrierung to datetime
df['abo_registrierung_min'] = pd.to_datetime(df['abo_registrierung_min'])
#df['abo_registrierung_min'] = df['abo_registrierung_min'].dt.date
df.abo_registrierung_min.describe()

In [ ]:
df.abo_registrierung_min.hist(bins=200,figsize=(11,6));
plt.xlabel('Time');
plt.ylabel('Count');

Due to the huge amount of unknown time stamps, which are set to 1900-01-01, this feature can not be used for the model. Therefore it should be dropped.

In [ ]:
df.abo_registrierung_min.hist(bins=200,figsize=(11,6));
plt.xlabel('Time');
plt.ylabel('Count');
plt.xlim('1960-01-01','2020-01-01');

### nl_registrierung_min

In [ ]:
df.nl_registrierung_min.isna().sum()

In [ ]:
# convert nl_registrierung to datetime
df['nl_registrierung_min'] = pd.to_datetime(df['nl_registrierung_min'], format='%Y-%m-%d')
#df['nl_registrierung_min'] = df['nl_registrierung_min'].dt.date
df.nl_registrierung_min.describe()

In [ ]:
df.info()

In [ ]:
df.nl_registrierung_min.head(2)

In [ ]:
df.nl_registrierung_min.hist(bins=200,figsize=(11,6));
plt.xlabel('Time');
plt.ylabel('Count');

We can observe a large number of unknown data which is set to 1900-01-01. This feature can therefore not be used in the model.

In [ ]:
df.nl_registrierung_min.hist(bins=200,figsize=(11,6));
plt.xlabel('Time');
plt.ylabel('Count');
plt.xlim('2000-01-01','2020-01-01')

In [ ]:
df.nl_registrierung_min.head(2)

### date_x

In [ ]:
df.date_x.isna().sum()

In [ ]:
# convert date_x to datetime
df['date_x'] = pd.to_datetime(df['date_x'], format='%Y-%m-%d')
df['date_x'] = df['date_x'].dt.date
df.date_x.describe()

In [ ]:
df.date_x.head(2)

In [ ]:
df.date_x.hist(bins=100,figsize=(11,6));
plt.xlabel('Time');
plt.ylabel('Count');

### kuendigungs_eingangs_datum

In [ ]:
df.kuendigungs_eingangs_datum.isna().sum()

In [ ]:
df.kuendigungs_eingangs_datum.fillna(value='1900-01-01 00:00:00',inplace=True);

In [ ]:
# convert date_x to datetime
df['kuendigungs_eingangs_datum'] = pd.to_datetime(df['kuendigungs_eingangs_datum'],errors='coerce',format='%Y-%m-%d')
#df['kuendigungs_eingangs_datum'] = df['kuendigungs_eingangs_datum'].dt.date
df.kuendigungs_eingangs_datum.describe()

In [ ]:
df.info()

In [ ]:
df.kuendigungs_eingangs_datum.hist(bins=200,figsize=(11,6));
plt.xlabel('Time');
plt.ylabel('Count');

In [ ]:
df.kuendigungs_eingangs_datum.hist(bins=1000,figsize=(11,6));
plt.xlabel('Time');
plt.ylabel('Count');
plt.xlim('2019-01-01','2020-06-01');
#plt.ylim(0,10000)

#### Kündigungen 2020

In [ ]:
kuendigungen_2020 = df[df['kuendigungs_eingangs_datum'] > '2020-01-01']
kuendigungen_2020.shape

In [ ]:
df['kuendigungs_eingangs_datum'][0]

In [ ]:
kuendigungen_2020.kuendigungs_eingangs_datum.max()

In [ ]:
kuendigungen_2020.kuendigungs_eingangs_datum.head()

In [ ]:
kuendigungen_2020 = kuendigungen_2020.sort_values(by="kuendigungs_eingangs_datum",ascending=True)

In [ ]:
kuendigungen_2020.head(2)

In [ ]:
import matplotlib.dates as mdates
from datetime import datetime
from matplotlib.dates import DateFormatter, MonthLocator, YearLocator

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')

fig, ax = plt.subplots(figsize=(18,8))

sns.countplot(kuendigungen_2020['kuendigungs_eingangs_datum']);
plt.xticks(rotation=90);
plt.xlabel('kuendigungs_eingang_datum 2020')

months = MonthLocator()
monthsFmt = DateFormatter("%b")

years = YearLocator()
yearsFmt = DateFormatter("%y")

ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))

#ax.xaxis.set_major_locator(months)
#ax.xaxis.set_major_formatter(monthsFmt)

#### Kündigungen 2019

In [ ]:
kuendigungen_2019 = df[(df['kuendigungs_eingangs_datum'] < '2020-01-01') & (df['kuendigungs_eingangs_datum'] > '2019-01-01')]
kuendigungen_2019.shape

In [ ]:
kuendigungen_2019.kuendigungs_eingangs_datum.min()

In [ ]:
kuendigungen_2019.kuendigungs_eingangs_datum.head()

In [ ]:
kuendigungen_2019 = kuendigungen_2019.sort_values(by="kuendigungs_eingangs_datum",ascending=True)

In [ ]:
kuendigungen_2019.head(2)

In [ ]:
import matplotlib.dates as mdates
from datetime import datetime
from matplotlib.dates import DateFormatter, MonthLocator, YearLocator

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')

fig, ax = plt.subplots(figsize=(18,8))

ax = sns.countplot(kuendigungen_2019['kuendigungs_eingangs_datum']);
plt.xticks(rotation=90);
plt.xlabel('kuendigungs_eingang_datum 2019')

months = MonthLocator()
monthsFmt = DateFormatter("%m-%d")

years = YearLocator()
yearsFmt = DateFormatter("%y")

#ax.xaxis.set_major_formatter(monthsFmt)
#ax.xaxis.set_major_locator(months)

date_form = DateFormatter("%m-%d")
#ax.xaxis.set_major_formatter(date_form)

# Ensure a major tick for each week using (interval=1) 
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))

#ax.set_xlim(pd.datetime.strptime(startDate,'%Y-%m-%d'),pd.datetime.strptime(stopDate,'%Y-%m-%d'))

### Time relation between kuendigungs_eingangs_datum und lieferstart

In [ ]:
df_churn = df.query('churn == 1')

In [ ]:
df_churn['delivery_to_churn'] = df_churn['kuendigungs_eingangs_datum'] - df_churn['liefer_beginn_evt']

In [ ]:
df_churn.head(2)

In [ ]:
df_churn['delivery_to_churn'].astype('timedelta64[s]').div(2592000).plot.hist(bins=120,figsize=(20,6));
plt.xlabel('months from delivery to churn receiving');
plt.xticks(np.arange(0,120,12));

In [ ]:
df_churn['delivery_to_churn'].astype('timedelta64[s]').div(2592000).max()

In [ ]:
df_churn['delivery_to_churn'] = df_churn['delivery_to_churn'].astype('timedelta64[s]').div(2592000)

In [ ]:
df_churn['delivery_to_churn'].head(2)

In [ ]:
months_bins = np.arange(0, 90.5, 0.5)
months_bins

In [ ]:
months_bin_series = pd.cut(df_churn['delivery_to_churn'], bins=months_bins, labels=months_bins[:-1])
months_bin_series.name = 'months_bins_2m'
df_churn['months_bins_2m'] = months_bin_series

In [ ]:
df_churn.head(2)

In [ ]:
import matplotlib.ticker as ticker
fig, ax = plt.subplots(figsize=(11,16))
df_churn.groupby('months_bins_2m').count()['churn'].plot(kind='bar',figsize=(20,6))
x_ticks = np.arange(0, 120, 2);
plt.xticks(x_ticks);
plt.xlim(0,98);
plt.ylabel('Number of churns');
plt.xlabel('Months from delivery to date of receipt of the churn');

#ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
#ax.xaxis.set_major_formatter(ticker.StrMethodFormatter("{x:.0f}"))

### Time Relation between kuendigungs_eingangs_datum and nl_registrierung_min

In [ ]:
df_churn['nl_start_to_churn'] = df_churn['kuendigungs_eingangs_datum'] - df_churn['nl_registrierung_min']

In [ ]:
df_churn.head(2)

In [ ]:
df_churn['nl_start_to_churn'].astype('timedelta64[s]').div(2592000).plot.hist(bins=1200,figsize=(20,6));
plt.xlabel('months from nl_registrierung min to churn receiving');
plt.xlim(-10,200);

In [ ]:
df_churn['nl_start_to_churn'] = df_churn['nl_start_to_churn'].astype('timedelta64[s]').div(2592000)

In [ ]:
df_churn['nl_start_to_churn'].head(2)

In [ ]:
months_bins = np.arange(0, 90.5, 0.5)
months_bins

In [ ]:
nl_min_bins_series = pd.cut(df_churn['nl_start_to_churn'], bins=months_bins, labels=months_bins[:-1])
nl_min_bins_series.name = 'nl_min_bins_05m'
df_churn['nl_start_to_churn_bins_05m'] = nl_min_bins_series

In [ ]:
fig, ax = plt.subplots(figsize=(11,16))
df_churn.groupby('nl_start_to_churn_bins_05m').count()['churn'].plot(kind='bar',figsize=(20,6))
x_ticks = np.arange(0, 120, 2);
plt.xticks(x_ticks);
plt.xlim(0,98);
plt.ylabel('Number of churns');
plt.xlabel('Months from nl_registrierung_min to date of receipt of the churn');

### Summary Time/Temporal Features

To sum up the EDA on the temporal features, the following was found:

- **lesedauer:** The lesedauer is an important measure for the churn probability: keep feature
- **liefer_beginn_evt:** This feature can be kept, since all users have this feature included: keep feature
- **abo_registrierung_min:** Due to the huge amount of unknown time stamps, which are set to 1900-01-01, this feature can not be used: drop feature
- **nl_registrierung_min:** Large number of unknown data: drop feature
- **date_x:** Is a random date or the kuendigungs_eingangs_datum, since we have to drop the kuendigungs_eingangs_datum, since it is directly related to the churn. 
- **kuendigungs_eingangs_datum:** must be dropped, since it is directly related to the churn date. 

## Activity features

These features were we can see interactions and communicate with the customers.

- newsletter
- clickrates
- openrates
- community

In [ ]:
index=0
for elem in df.columns:   
    print(f"{index} {elem}")
    index+=1

Create dataframes for differen activity features to plot and describe them one by one.

In [ ]:
df_zon = df.iloc[::, 21:35]                # zones are special areas that need registration
df_cnt = df.iloc[::, 35:40]                # cnt is the number of subscribtions the contract holds (families, libaries etc.)
df_nl = df.iloc[::, 41:51]                 # newsletter drop technical details
df_nl.drop(["nl_blacklist_sum", "nl_bounced_sum", "nl_sperrliste_sum", "nl_opt_in_sum", "nl_fdz_organisch", "nl_registrierung_min"], axis=1, inplace=True)
df_reg = df.iloc[::, 51:55]                # registration needed (for accses to these services)
df_nl_interact = df.iloc[::, 55:77]        # newsletter interactions
df_nl_bestandskunden = df.iloc[::, 77:99]  # newsletter existing customers
df_nl_produktnews = df.iloc[::, 99:121]    # productnews (kind of newsletter but more commercial)
df_nl_hamburg = df.iloc[::, 121:143]       # newsletter region hamburg
df_zb = df.iloc[::, 143:165]               # zb = zeitbrief kind of letter

### zon features

In [ ]:
for elem in df_zon.columns[:2]:
    describe_frame = pd.DataFrame(round(df[elem].describe(),1))
    ax = sns.countplot(x=elem, data=df, hue="churn")
    ax.set(xlabel=elem, ylabel="Frequency", xlim=[-0.5, 2.5])
    plt.title(f"Binned distribution of {elem}")
    plt.show()
    print(describe_frame.T)
    print("\n")

In [ ]:
eda.correlogram(df_zon)

**Observation:** All the features in the list have a range between 0 and 2. 0 is a code for no interactions, 1 describes "registration startetd", 2 is "registration completetd". As you can see in the plots before these comments, the rates of customers to even register or complet register is very low. The correlogram(df_zon) showed no mentionable correlations (rounded values -1 or 0).

### cnt features

In [ ]:
df_cnt.sample(5)

In [ ]:
eda.correlogram(df_cnt)

**Observation:** The cnt features contain information of how many and which subscribtion types are held by the "auftrag_new_id". As we can see above, the column "cnt_abo" holds the sum of the other columns. The last colomn is holds information about how many of these are "conversion" by trial subscription. When we had a look at the correlogram(df_cnt) there were strong correlation. This is surprising cause the cnt_abo features are these that count the sum of abos a customers keeps. For example if one person has more than one newspaper, or a newspaper and a magazin, or newspaper print and digital. For more information about the subscription types check the [wiki: Business understanding](https://github.com/jb-ds2020/nf-ds3-capstone-churn-prevention/wiki/Business-understanding-(publishing-and-news)).


### reg features

In [ ]:
df_reg.boa_reg.value_counts()

In [ ]:
for elem in df_reg.columns:
    describe_frame = pd.DataFrame(round(df[elem].describe(),1))
    ax = sns.countplot(x=elem, data=df, hue="churn")
    ax.set(xlabel=elem, ylabel="Frequency", xlim=[-0.5, 1.5])
    plt.title(f"Binned distribution of {elem}")
    plt.show()
    print(describe_frame.T)
    print("\n")

In [ ]:
eda.correlogram(df_reg)

**Observations:** These features contain information, if clients have registred for special areas of the homepage. If it is 1 they have if not it is 0. As you can see most users, are not interessted in these areas and services. The restrictad hp areas (registration needed) show correlations between each other between 15 and 27.

### Customer Email
The feature email_am_kunden is a feature which shows if the email of an subscriber is known to the publisher.
* 0:not known
* 1:known

In [ ]:
plt.subplots(figsize=(10,6))
ax = sns.countplot(x='email_am_kunden', hue='churn', data=df)
ax.set(xlabel='Email am Kunden', ylabel='Frequency')
ax.set(ylim=(0, None))


ax.set_xticklabels(ax.get_xticklabels(), rotation=90);

In [ ]:
email_am_kunden_churn = crosstab_evaluation(df.email_am_kunden,df.churn)
crosstab_barplot(email_am_kunden_churn,['no churn','churn'],xlabelname='Email am Kunden')

In [ ]:
df['email_am_kunden'].value_counts()/df.shape[0]

__Observations:__ 
* Only 6.4% of the mail adresses of all subscribers are unknown
* The relative churn rate is a little bit higher for subscibers with known email adresses, but not a big difference is visible

### Newsletter features

In [ ]:
eda.correlogram(df_nl_interact)

**Observations:** df_nl the holds the four feature "nl_zeitbrief", "nl_zeitshop", "nl_zeitvrlag_hamburg" and "nl_aktivitaet". The only correlation is between "nl_aktivitaet" and "nl_zeitbrief". Altought this is not surprising, cause the feature "nl_aktivitaet" is an aggregation of user activity on the newsletters.

### Shop Kauf

In [ ]:
plt.subplots(figsize=(20,6))
ax = sns.countplot(x='shop_kauf', hue='churn', data=df)
ax.set(ylabel='Frequency')
ax.set(ylim=(0, 400))
plt.title('Shop Kauf', fontsize=22)
L=plt.legend(fontsize=20,loc=(1.04,0.83))
L.get_texts()[0].set_text(labellist[0])
L.get_texts()[1].set_text(labellist[1])

ax.set_xticklabels(ax.get_xticklabels(), rotation=90);


In [ ]:
shop_churn = crosstab_evaluation(df.shop_kauf,df.churn)
crosstab_barplot(shop_churn,['no churn','churn'],xlabelname='Shop Kauf')
plt.xlim(-0.5,25.5);

### Newsletter
__Email-Newsletter:__ <br>
with: 0:not available, 1: unsubscribed, 2: active
* nl_zeitbrief 
* nl_zeitshop
* nl_zeitverlag_hamburg
* nl_fdz_organisch: on organic recipient list Loyaltyprogram fdZ
</p>
* Blacklist: Number of blacklist entries
* Bounced: Number of Bounces
* Aktivität: Number on how many newsletters in the house
* Sperrliste
</p>
* nl_opt_in_sum: Number of Optins

In [ ]:
title = ['Zeitbrief', 'Zeitshop', "Zeitverlag Hamburg", 'FDZ organisch']
labellist = ['no churn','churn']

for i, nl in enumerate(['nl_zeitbrief', 'nl_zeitshop', 'nl_zeitverlag_hamburg', 'nl_fdz_organisch']):
    
    
    plt.subplots(figsize=(10,6))
    ax = sns.countplot(x=nl, hue='churn', data=df)
    ax.set(xlabel='Email Newsletter', ylabel='Frequency')
    plt.title(title[i], fontsize=22)
    L=plt.legend(fontsize=20,loc=(1.04,0.83))
    L.get_texts()[0].set_text(labellist[0])
    L.get_texts()[1].set_text(labellist[1])

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

In [ ]:
for i in ['nl_zeitbrief', 'nl_zeitshop', 'nl_zeitverlag_hamburg', 'nl_fdz_organisch']:
    x = crosstab_evaluation(df[i],df.churn)
    crosstab_barplot(x,['no churn','churn'],xlabelname=i)

__Absolute Blacklist and Bounced__

In [ ]:
title = ['Blacklist', 'Bounced']
labellist = ['no churn','churn']
    
for i, nl in enumerate(['nl_blacklist_sum', 'nl_bounced_sum']):
    
    
    plt.subplots(figsize=(10,6))
    ax = sns.countplot(x=nl, hue='churn', data=df)
    ax.set(xlabel='Quantity', ylabel='Frequency')
    plt.title(title[i], fontsize=22)
    L=plt.legend(fontsize=20,loc=(1.04,0.83))
    L.get_texts()[0].set_text(labellist[0])
    L.get_texts()[1].set_text(labellist[1])

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

__Relative Blacklist and Bounced__

In [ ]:
for i in ['nl_blacklist_sum', 'nl_bounced_sum']:
    x = crosstab_evaluation(df[i],df.churn)
    crosstab_barplot(x,['no churn','churn'],xlabelname=i)

__Absolute NL Aktivitaet and Sperrliste__

In [ ]:
title = ['Activity', 'Blocking list']
labellist = ['no churn','churn']
    
for i, nl in enumerate(['nl_aktivitaet', 'nl_sperrliste_sum']):
    
    
    plt.subplots(figsize=(10,6))
    ax = sns.countplot(x=nl, hue='churn', data=df)
    ax.set(xlabel='Quantity', ylabel='Frequency')
    plt.title(title[i], fontsize=22)
    L=plt.legend(fontsize=20,loc=(1.04,0.83))
    L.get_texts()[0].set_text(labellist[0])
    L.get_texts()[1].set_text(labellist[1])

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

__Relative NL Aktivität and Sperrliste__

In [ ]:
for i in ['nl_aktivitaet', 'nl_sperrliste_sum']:
    x = crosstab_evaluation(df[i],df.churn)
    crosstab_barplot(x,['no churn','churn'],xlabelname=i)

__Absolute NL Optins__

In [ ]:
plt.subplots(figsize=(10,6))
ax = sns.countplot(x=df['nl_opt_in_sum'], hue='churn', data=df)
ax.set(xlabel='Quantity', ylabel='Frequency', xlim=[0,None])
plt.title('NL Optins', fontsize=22)
L=plt.legend(fontsize=20,loc=(1.04,0.83))
L.get_texts()[0].set_text(labellist[0])
L.get_texts()[1].set_text(labellist[1])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90); 

__Relative NL Optins__

In [ ]:
x = crosstab_evaluation(df['nl_opt_in_sum'],df.churn)
crosstab_barplot(x,['no churn','churn'],xlabelname='NL Optins')

#### Bestandskunden
* with received, opened, clicked, unsubscribed mails
* time period = 6m 

In [ ]:
title = ['Received Bestandskunden (6m)', 'Opened Bestandskunden (6m)', 'Clicked Bestandskunden (6m)', 'Unsubscribed Bestandskunden (6m)', 'Clickrate Bestandskunden (3m)', 'Openrate Bestandskunden (3m)']
labellist = ['no churn','churn']
    
for i, nl in enumerate(['received_anzahl_bestandskunden_6m', 'openedanzahl_bestandskunden_6m', 'clicked_anzahl_bestandskunden_6m', 'unsubscribed_anzahl_bestandskunden_6m', 'clickrate_bestandskunden_3m', 'openrate_bestandskunden_3m']):
    
    
    plt.subplots(figsize=(10,6))
    ax = sns.countplot(x=nl, hue='churn', data=df)
    ax.set(xlabel='Quantity', ylabel='Frequency')
    plt.title(title[i], fontsize=22)
    L=plt.legend(fontsize=20,loc=(1.04,0.83))
    L.get_texts()[0].set_text(labellist[0])
    L.get_texts()[1].set_text(labellist[1])

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

In [ ]:
for i in ['received_anzahl_bestandskunden_6m', 'openedanzahl_bestandskunden_6m', 'clicked_anzahl_bestandskunden_6m', 'unsubscribed_anzahl_bestandskunden_6m', 'clickrate_bestandskunden_3m', 'openrate_bestandskunden_3m']:
    
    x = crosstab_evaluation(df[i],df.churn)
    crosstab_barplot(x,['no churn','churn'],xlabelname=i)
   

#### Produktnews
* with received, opened, clicked, unsubscribed mails
* time period = 6m 

In [ ]:
title = ['Received Produktnews (6m)', 'Opened Produktnews (6m)', 'Clicked Produktnews (6m)', 'Unsubscribed Produktnews (6m)', 'Clickrate Produktnews (3m)', 'Openrate Produktnews (3m)']
labellist = ['no churn','churn']
    
for i, nl in enumerate(['received_anzahl_produktnews_6m', 'openedanzahl_produktnews_6m', 'clicked_anzahl_produktnews_6m', 'unsubscribed_anzahl_produktnews_6m', 'clickrate_produktnews_3m', 'openrate_produktnews_3m']):
    
    
    plt.subplots(figsize=(10,6))
    ax = sns.countplot(x=nl, hue='churn', data=df)
    ax.set(xlabel='Quantity', ylabel='Frequency')
    ax.set(ylim=(0, 5000))
    plt.title(title[i], fontsize=22)
    L=plt.legend(fontsize=20,loc=(1.04,0.83))
    L.get_texts()[0].set_text(labellist[0])
    L.get_texts()[1].set_text(labellist[1])

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

In [ ]:
for i in ['received_anzahl_produktnews_6m', 'openedanzahl_produktnews_6m', 'clicked_anzahl_produktnews_6m', 'unsubscribed_anzahl_produktnews_6m', 'clickrate_produktnews_3m', 'openrate_produktnews_3m']:
    x = crosstab_evaluation(df[i],df.churn)
    crosstab_barplot(x,['no churn','churn'],xlabelname=i)

#### Hamburg
Daily Newsletter (Mo-Fr)
* with received, opened, clicked, unsubscribed mails
* time period = 6m 

In [ ]:
plt.subplots(figsize=(10,6))
ax = sns.countplot(x='received_anzahl_hamburg_6m', hue='churn', data=df)
ax.set(xlabel='Quantity', ylabel='Frequency')
ax.set(ylim=(0, 20))
ax.set(xlim=(0, 10))
plt.title('Recieved Hamburg (6m)', fontsize=22)
L=plt.legend(fontsize=20,loc=(1.04,0.83))
L.get_texts()[0].set_text(labellist[0])
L.get_texts()[1].set_text(labellist[1])

ax.set_xticklabels(ax.get_xticklabels(), rotation=90);

In [ ]:
title = ['Opened Hamburg (6m)', 'Clicked Hamburg (6m)', 'Clickrate Hamburg (3m)', 'Openrate Hamburg (3m)']
labellist = ['no churn','churn']
    
for i, nl in enumerate(['openedanzahl_hamburg_6m', 'clicked_anzahl_hamburg_6m', 'clickrate_hamburg_3m', 'openrate_hamburg_3m']):
    
    
    plt.subplots(figsize=(10,6))
    ax = sns.countplot(x=nl, hue='churn', data=df)
    ax.set(xlabel='Quantity', ylabel='Frequency')
    ax.set(ylim=(0, 400))
    ax.set(xlim=(0, 10))
    plt.title(title[i], fontsize=22)
    L=plt.legend(fontsize=20,loc=(1.04,0.83))
    L.get_texts()[0].set_text(labellist[0])
    L.get_texts()[1].set_text(labellist[1])

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

In [ ]:
plt.subplots(figsize=(10,6))
ax = sns.countplot(x='unsubscribed_anzahl_hamburg_6m', hue='churn', data=df)
ax.set(xlabel='quantity', ylabel='Frequency')
ax.set(ylim=(0, 400))
plt.title('Unsubscribed Hamburg (6m)', fontsize=22)
L=plt.legend(fontsize=20,loc=(1.04,0.83))
L.get_texts()[0].set_text(labellist[0])
L.get_texts()[1].set_text(labellist[1])


ax.set_xticklabels(ax.get_xticklabels(), rotation=90);

In [ ]:
for i in ['received_anzahl_hamburg_6m', 'openedanzahl_hamburg_6m', 'clicked_anzahl_hamburg_6m', 'unsubscribed_anzahl_hamburg_6m', 'clickrate_hamburg_3m', 'openrate_hamburg_3m']:
    x = crosstab_evaluation(df[i],df.churn)
    crosstab_barplot(x,['no churn','churn'],xlabelname=i, figsize_x=25)

#### Zeitbrief
* with received, opened, clicked, unsubscribed mails
* time period = 6m  

In [ ]:
title = ['Received Zeitbrief (6m)', 'Opened Zeitbrief (6m)', 'Clicked Zeitbrief (6m)', 'Unsubscribed Zeitbrief (6m)', 'Clickrate Zeitbrief (3m)', 'Openrate Zeitbrief (3m)']
labellist = ['no churn','churn']
    
for i, nl in enumerate(['received_anzahl_zeitbrief_6m', 'openedanzahl_zeitbrief_6m', 'clicked_anzahl_zeitbrief_6m', 'unsubscribed_anzahl_zeitbrief_6m', 'clickrate_zeitbrief_3m', 'openrate_zeitbrief_3m']):
    
    
    plt.subplots(figsize=(15,6))
    ax = sns.countplot(x=nl, hue='churn', data=df)
    ax.set(xlabel='Quantity', ylabel='Frequency')
    ax.set(ylim=(0, 5000))
    plt.title(title[i], fontsize=22)
    L=plt.legend(fontsize=20,loc=(1.04,0.83))
    L.get_texts()[0].set_text(labellist[0])
    L.get_texts()[1].set_text(labellist[1])

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

In [ ]:
for i in ['received_anzahl_zeitbrief_6m', 'openedanzahl_zeitbrief_6m', 'clicked_anzahl_zeitbrief_6m', 'unsubscribed_anzahl_zeitbrief_6m', 'clickrate_zeitbrief_3m', 'openrate_zeitbrief_3m']:
    x = crosstab_evaluation(df[i],df.churn)
    crosstab_barplot(x,['no churn','churn'],xlabelname=i)

### Newsletter Email - Carlotta
Subscribers which are subscribed to an Newsletter online will be send emials. The number depends on the frequency of delivery of the newsletter. The Newsletter _Hamburg_ and the Newsletter _Zeitbiref_ are for example daily (Mo-Fr) Newsletter. For the purpose of our analysis it is only important if the subscriber becomes Email-Newsletter or not. <br>
Therefore, the following section is going to flatten thos columns into 1 and 0. 
* df_nl_bestandskunden
* df_nl_produktnews
* df_nl_hamburg
* df_nl_zeitbrief

    - received
    - opened quantity
    - clicked quantity
    - unsubscribed

In [ ]:
df_nl_bestandskunden_1 = df.iloc[::, 77:93]  # newsletter existing customers without rates
df_nl_produktnews_1 = df.iloc[::, 99:115]    # productnews (kind of newsletter but more commercial)without rates
df_nl_hamburg_1 = df.iloc[::, 121:137]       # newsletter region hamburg without rates
df_zb_1 = df.iloc[::, 143:159]               # newsletter zeitbrief without rates

__Typos in the colum names corrected__

In [ ]:
df.rename(columns={'openedanzahl_bestandskunden_6m': 'opened_anzahl_bestandskunden_6m',
                   'openedanzahl_produktnews_6m': 'opened_anzahl_produktnews_6m',
                   'openedanzahl_hamburg_6m': 'opened_anzahl_hamburg_6m',
                   'openedanzahl_zeitbrief_6m': 'opened_anzahl_zeitbrief_6m'}, inplace=True)

In [ ]:
for i in df_nl_bestandskunden_1:
    df[i] = df[i].apply(flatten_greater_0)
    
for i in df_nl_produktnews_1:
    df[i] = df[i].apply(flatten_greater_0)

for i in df_nl_hamburg_1:
    df[i] = df[i].apply(flatten_greater_0)
    
for i in df_zb_1:
    df[i] = df[i].apply(flatten_greater_0)

Furthermore we can obsereve the same pattern over all the different Newsletter: Only the ones which are not subscribed to an newsletter churn. Therefore the aggregation of the Newsletter is beeing considert. 

In [ ]:
name = ['received_anzahl', 'opened_anzahl', 'clicked_anzahl', 'unsubscribed_anzahl']
art = ['bestandskunden','produktnews','hamburg','zeitbrief']
zeitraum = ['1w', '1m', '3m', '6m']
titel = ['nl_received_1w', 'nl_received_1m', 'nl_received_3m', 'nl_received_6m', 'nl_opened_1w', 'nl_opened_1m', 'nl_opened_3m', 
        'nl_opened_6m','nl_clicked_1w', 'nl_clicked_1m', 'nl_clicked_3m', 'nl_clicked_6m', 'nl_unsubscribed_1w', 'nl_unsubscribed_1m', 
        'nl_unsubscribed_3m', 'nl_unsubscribed_6m'] 

In [ ]:
links = []
for n in name:
    for z in zeitraum:
        for a in art:
            links.append(n + '_' + a + '_' + z)

In [ ]:
for t in titel:
    df[t] = df[links[0]] + df[links[1]] + df[links[2]] + df[links[3]]
    links = links[3:]

__Aggregated Columns:__

In [ ]:
title = ['Received NL (6m)', 'Opened NL (6m)', 'Clicked NL (6m)', 'Unsubscribed NL (6m)']
labellist = ['no churn','churn']
    
for i, nl in enumerate(['nl_received_6m', 'nl_opened_6m', 'nl_clicked_6m', 'nl_unsubscribed_6m']):
    
    
    plt.subplots(figsize=(10,6))
    ax = sns.countplot(x=nl, hue='churn', data=df)
    ax.set(xlabel='Quantity', ylabel='Frequency')
    ax.set(ylim=(0, None))
    plt.title(title[i], fontsize=22)
    L=plt.legend(fontsize=20,loc=(1.04,0.83))
    L.get_texts()[0].set_text(labellist[0])
    L.get_texts()[1].set_text(labellist[1])

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

### Clickrate and Openrate Features - Carlotta

In [ ]:
def flatten_rate(flat):
    if 0.75 < flat < 1:
        return 1
    elif 0 < flat < 0.25:
        return 0.25
    elif 0.25 < flat < 0.5:
        return 0.5
    elif 0.5 < flat < 0.75:
        return 0.75
    elif flat > 1:
        return 1.5
    else:
        return 0

In [ ]:
for i in ['openrate_1w', 'clickrate_1w', 'openrate_1m', 'clickrate_1m', 'openrate_3m', 'clickrate_3m']:
    df[i] = df[i].apply(flatten_rate)

In [ ]:
for i in ['openrate_1w', 'clickrate_1w', 'openrate_1m', 'clickrate_1m', 'openrate_3m', 'clickrate_3m']:
    

    plt.subplots(figsize=(20,6))
    ax = sns.countplot(x=i, hue='churn', data=df)
    ax.set(xlabel=i, ylabel='Frequency')

    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

__Observations:__ To have an better overview of the clickrates we binned them into manageables rates (0,0.25,0.5,0.75,1 and 1.5 as a representative of all rates bigger than 1)

## Target Variable Analysis

Our target variable is "churn", which is 1 - churn or 0 - no churn. 

In [ ]:
## Show the imbalance of churn target
df.churn.value_counts().plot(kind='bar',figsize=(11,6));
plt.ylabel('Count');
#plt.xlabel('Churn')
plt.xticks(np.arange(2), ('no churn', 'churn'),fontsize=20,rotation=0);

## Feature Analysis

In [ ]:
df.drop('churn', axis=1).corrwith(df.churn).sort_values().plot(kind='barh',figsize=(10, 50));

## Export the dataframe to .csv file

## Conclusion

# Feature Engineering

## zon features

In [ ]:
len(df_zon.columns)

The 14 features in this list will be transformed. We will use the information only if the registration is completet (value = 2) and than we will aggregate them to get a continues feature "sum_zon" that keeps the information how many users registred for the zones and how many.

In [ ]:
def flatten_greater_1(flat):
    if flat > 1:
        return 1
    else:
        return 0

In [ ]:
print("Sum zon features")
for i in df_zon:
    df[i] = df[i].apply(flatten_greater_1)
sum_zon = df_zon.sum(axis=1)
print(sum_zon.sample(5))
print("there are a few customers registred or active in more than one")

In [ ]:
sum_zon = sum_zon.to_frame(name="sum_zon")
sum_zon.head(1)

In [ ]:
df = df.join(sum_zon)

In [ ]:
print(df.sum_zon.value_counts())
ax = sns.countplot(x=df.sum_zon, data=df, hue="churn")
ax.set(xlabel=elem, ylabel="Frequency", xlim=[-0.5, 5.5])
plt.title(f"Binned distribution of {elem}")
plt.show()

## reg features

In [ ]:
len(df_reg.columns)

In [ ]:
eda.meta(df_reg)

The 4 features in this list will be aggregated to a feature "sum_reg" where we will find the sum of how many registred areas the user is registred to.

In [ ]:
print("Sum registrated for special areas features")
sum_reg = df_reg.sum(axis=1)
print(sum_reg.sample(10))
print("there are a few customers registred or active in more than one")

In [ ]:
sum_reg = sum_reg.to_frame(name="sum_reg")
sum_reg.head(1)

In [ ]:
df = df.join(sum_reg)

In [ ]:
print(df.sum_reg.value_counts())
ax = sns.countplot(x=df.sum_reg, data=df, hue="churn")
ax.set(xlabel=elem, ylabel="Frequency", xlim=[-0.5, 5.5])
plt.title(f"Binned distribution of {elem}")
plt.show()

## Newsletter Flattening

In [ ]:
# Carlotta: flattening of newletter data, sum and join.

## Time Features

In [ ]:
# - abo_registrierung_min_()

## Dropping 

In [ ]:
df.head(1)

In [ ]:
droppinglist_obvious = ['training_set','avg_churn']

### Dropping NL mails - Carlotta
In the follwoing section we drop the columns of the different newsletter mail activities, because we created some aggregated new columns consisting of those. Therefore these columns are redundant.

In [ ]:
droppinglist_tech = ["nl_blacklist_sum", 
                     "nl_bounced_sum", 
                     "nl_sperrliste_sum", 
                     "nl_opt_in_sum", 
                     "nl_fdz_organisch", 
                     "nl_registrierung_min"]

In [ ]:
droppinglist_nl_mail = df_nl_bestandskunden_1.columns.values.tolist() + df_nl_produktnews_1.columns.values.tolist() + df_nl_hamburg_1.columns.values.tolist() + df_zb_1.columns.values.tolist()

In [ ]:
# drop ist weiter unten

### Dropping cnt, zon, reg

In [ ]:
droppinglist_cnt = list(df_cnt.columns[1:])
droppinglist_zon = list(df_zon.columns)
droppinglist_reg = list(df_reg.columns)

### Dropping Time and Customer/personal features

In [ ]:
# time feature dropping 
droppinglist_time = ['abo_registrierung_min','nl_registrierung_min','date_x','kuendigungs_eingangs_datum']

In [ ]:
# we want to include only the best geographical information, so we drop plz_1, plz_2 and keep plz_3. ort is therefore also deleted. 
droppinglist_geo = ['plz_1','plz_2','ort'] # maybe keep ort instead of plz due to the higher information for abroad cities. 

### Drop

In [ ]:
print(f"Number of features {df.shape[1]}")
df = df.drop(droppinglist_obvious, axis=1)
print(f"Number of features {df.shape[1]}")
df = df.drop(droppinglist_nl_mail, axis=1)
print(f"Number of features {df.shape[1]}")
df = df.drop(droppinglist_tech, axis=1)
print(f"Number of features {df.shape[1]}")
df = df.drop(droppinglist_cnt, axis=1)
print(f"Number of features {df.shape[1]}")
df = df.drop(droppinglist_zon, axis=1)
print(f"Number of features {df.shape[1]}")
df = df.drop(droppinglist_reg, axis=1)
print(f"Number of features {df.shape[1]}")
# added time and geo
df = df.drop(droppinglist_time, axis=1)
print(f"Number of features {df.shape[1]}")
df = df.drop(droppinglist_geo, axis=1)
print(f"Number of features {df.shape[1]}")

## Conclusion and Comparison before after dropping and feature engineering

## Export final Dataframe

# Predictive Modelling

## Structure of the notebooks for machine learning

We used different jupyter notebook for different methods of machine learning. These notebooks are linked here:
    
- ...

- ...

- ...

- ...

# Data Visualization

The results and our main finding can be found in our presentation:

# Future Work

The aspects of future work can be found in our presentation. 